In [1]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

## Считывание данных

In [2]:
full_data = pd.read_csv('rating.csv')
full_data = full_data.sort_values(by=['userId','timestamp']).reset_index(drop=True)
full_data

,userId,movieId,rating,timestamp
0,1,924,3.5,2004-09-10 03:06:38
1,1,919,3.5,2004-09-10 03:07:01
2,1,2683,3.5,2004-09-10 03:07:30
3,1,1584,3.5,2004-09-10 03:07:36
4,1,1079,4.0,2004-09-10 03:07:45
...,...,...,...,...
20000258,138493,6534,3.0,2009-12-07 18:18:28
20000259,138493,53464,4.0,2009-12-07 18:18:40
20000260,138493,1275,3.0,2010-01-01 20:42:32
20000261,138493,6996,3.0,2010-01-01 20:42:35


In [3]:
#удаляются из рассмотрения оценки тех фильмов, количество оценок которых суммарно меньше порога 
#а также пользователи, у которых в сумме меньше порога оценок
 
def filter_data(full_data, min_cnt_marks_user=100, min_cnt_marks_movie = 10):
    
    cnts_movies = full_data[['movieId','rating']].groupby(by=['movieId']).size()
    full_data = full_data[full_data.movieId.isin(cnts_movies[cnts_movies > min_cnt_marks_movie].index)]
    cnts_users_marks = full_data[['userId']].groupby(by=['userId']).size().sort_values(ascending=False)
    full_data = full_data[full_data.userId.isin(cnts_users_marks[cnts_users_marks > min_cnt_marks_user].index)]
    
    return full_data

In [4]:
full_data = filter_data(full_data, min_cnt_marks_user=100, min_cnt_marks_movie = 10)

In [5]:
print(f'Всего пользователей: {full_data.userId.nunique()}') # всего пользователей
print(f'Всего фильмов: {full_data.movieId.nunique()}') #всего фильмов

Всего пользователей: 51852
Всего фильмов: 15079


## Вспомогательная функция

In [6]:
#вспомогательная функция для информации о тесте и трейне
def info_train_test(data_train, data_test):
    full_data = pd.read_csv('rating.csv')
    
    count_new_users = 0
    test_unique_users = data_test.userId.unique()
    train_unique_users = data_train.userId.unique()
    ids_new_users = []


    count_new_movies = 0
    test_unique_movies = data_test.movieId.unique()
    train_unique_movies = data_train.movieId.unique()
    ids_new_movies = []

    for movie in test_unique_movies:
        if movie not in train_unique_movies:
            count_new_movies += 1
            ids_new_movies.append(movie)
    print(f'Новых фильмов в тесте: {count_new_movies}')

    print(f' Новых фильмов в тесте: {round((count_new_movies/full_data.movieId.nunique()) * 100)} % от всех')

## Метрика и разбиение для валидации

### Выбор метрики

Метрика, по которой будет оцениваться модель должна отвечать запросам бизнеса, так как в задаче нет конкретных бизнес-требований, то остается предположить какие задачи будущая рекомендательная система должна решать. Если предполагать, что задача заключается в том, чтобы пользователь заходя на сервис, не тратил много времени в поисках фильма для просмотра и не покинул сервис, а сразу же получал в рекомендации фильм, который он захочет посмотреть, то рекомендации должны быть наиболее релевантны и в первую очередь предлагаться должны наиболее подходящие фильмы. Для оценки релевантности рекомендаций в качестве метрики для модели, я считаю целесообразным выбрать nDCG@k, эта метрика позволяет оценить релевантность рекомендаций, который получает пользователь, а также учитывает позиции релевантных объектов. Для того, чтобы рассчитывать данную метрику можно использовать следующую стратегию разбиения на train/test: для каждого пользователя выбрать N случайных фильмов, которым он поставил оценку, все остальные фильмы отправить в трейн. таким образом для каждого пользователя будет сформирован список из N фильмов, которые будут считаться релевантными. Затем после составления рекомендаций для каждого пользователя из M фильмов, отметить как 0 нерелевантные фильмы и как 1-релевантные, получим список типа [0, 1, 0, 0, 0, ... ] и после этого рассчитывается метрика nDCG@M для каждого пользователя, затем усредняется по всем пользователям.

### Разбиение на train/test

Для разбивки на трейн/тест берутся 30 случайных оценок фильмов каждого пользователя. Используя такую разбивку, по сути, мы делаем вид, что пользователи поставили эти 30 оценок после самой последней оценки трейна и если предположить, что вкусы пользователей не меняется, то небольшое заглядывание в будущее (имется в виду тот факт, что оценка фильму в тесте могла быть поставлена раньше других оценок в трейне, то есть на тот момент были еще неизвестны оценки, на основании которых в конечном итоге строится рекомендация) не является критичным.

In [7]:
data_test = full_data.groupby(by='userId').sample(n=30, random_state=1)
data_test

,userId,movieId,rating,timestamp
93,1,4011,4.0,2005-04-02 23:43:26
114,1,8368,4.0,2005-04-02 23:48:08
19,1,1219,4.0,2004-09-10 03:13:14
69,1,1262,3.5,2005-04-02 23:32:15
53,1,50,3.5,2005-04-02 23:29:40
...,...,...,...,...
20000190,138493,8605,4.5,2009-11-09 16:52:04
20000074,138493,1407,3.5,2009-10-17 22:08:43
20000168,138493,594,4.0,2009-10-28 17:21:35
20000152,138493,50872,3.5,2009-10-28 17:19:48


In [8]:
data_train = full_data.merge(data_test, how='left', on=['timestamp', 'userId','movieId'])
data_train = data_train[(data_train.rating_x > 0) 
                        & ((data_train.rating_y > 0)==False)].drop(['rating_y'], axis=1).rename(columns={'rating_x':'rating'})

In [9]:
data_test.to_csv('rand_test_30.csv', sep=',', index=False)
data_train.to_csv('rand_train_30.csv', sep=',', index=False)